<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/PIXEL-PIONEERS-TUTORIALS/blob/main/biprinting_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stable-baselines3[extra] gymnasium matplotlib

In [ ]:
!pip install streamlit

In [ ]:
# Cell 1: Create organ_bioprint_env.py
%%writefile organ_bioprint_env.py
import numpy as np
import gymnasium as gym
from gymnasium import spaces

class OrganBioprintEnv(gym.Env):
    """ Detailed environment for 3D bioprinting of a heart model. """
    def __init__(self, resolution=64, deposition_rate=0.1, velocity_scale=10.0):
        super().__init__()
        self.resolution = resolution
        self.deposition_rate = deposition_rate
        self.velocity_scale = velocity_scale
        self.material_types = 3  # Myocardium, Endocardium, Epicardium
        self.bioprint_env = np.zeros((resolution, resolution, resolution, self.material_types), dtype=np.float32)
        self.observation_space = spaces.Box(low=0, high=1, shape=(resolution, resolution, resolution, self.material_types), dtype=np.float32)
        self.action_space = spaces.Box(low=0, high=1, shape=(6,), dtype=np.float32)  # Including an extrusion parameter
        self.target_structure = self.generate_target_structure()
        self.print_head_position = np.array([resolution // 2, resolution // 2, resolution // 2], dtype=np.float32)

    def generate_target_structure(self):
        target = np.zeros((self.resolution, self.resolution, self.resolution, self.material_types), dtype=np.float32)
        r = self.resolution // 2
        grid = np.indices((self.resolution, self.resolution, self.resolution))
        distance = np.sqrt((grid[0] - r)**2 + (grid[1] - r)**2 + (grid[2] - r)**2)
        target[distance < r * 0.5, 0] = 1
        target[(distance >= r * 0.5) & (distance < r * 0.75), 1] = 1
        target[distance >= r * 0.75, 2] = 1
        return target

    def step(self, action):
        dx, dy, dz, v_adjust, m_select, extrude = action
        m_select = int(np.clip(np.round((m_select + 1) * 0.5 * (self.material_types - 1)), 0, self.material_types - 1))
        self.print_head_position += np.array([dx, dy, dz], dtype=np.float32) * self.velocity_scale
        self.print_head_position = np.clip(self.print_head_position, 0, self.resolution - 1)
        x, y, z = self.print_head_position.astype(int)
        if extrude > 0.5:  # Extrusion action
            self.bioprint_env[x, y, z, m_select] += self.deposition_rate * extrude
            self.bioprint_env = np.clip(self.bioprint_env, 0, 1)

        reward = -np.sum(np.abs(self.bioprint_env - self.target_structure))
        done = np.array_equal(self.bioprint_env, self.target_structure)
        info = {"position": self.print_head_position.tolist(), "selected_material": m_select}
        return self.bioprint_env.copy(), reward, done, False, info

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.bioprint_env = np.zeros((self.resolution, self.resolution, self.resolution, self.material_types), dtype=np.float32)
        self.print_head_position = np.array([self.resolution // 2, self.resolution // 2, self.resolution // 2], dtype=np.float32)
        return self.bioprint_env.copy(), {}

Overwriting organ_bioprint_env.py


In [ ]:
# Cell 2: Create train_model.py
%%writefile train_model.py
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from organ_bioprint_env import OrganBioprintEnv

def train_model(total_timesteps=20000, save_path="bioprint_heart_model.zip"):
    env = OrganBioprintEnv()
    env = DummyVecEnv([lambda: env])

    model = PPO("MlpPolicy", env, verbose=1)
    model.learn(total_timesteps=total_timesteps)
    model.save(save_path)
    env.close()
    print(f"Model training complete. Saved to {save_path}")

if __name__ == "__main__":
    train_model()

Writing train_model.py


In [ ]:
# Cell 3: Create bioprinting_app.py
%%writefile bioprinting_app.py
import streamlit as st
import numpy as np
import plotly.graph_objects as go
from stable_baselines3 import PPO
from organ_bioprint_env import OrganBioprintEnv

model_path = "bioprint_heart_model.zip"
model = PPO.load(model_path)
env = OrganBioprintEnv()

def simulate_step(action):
    obs, rewards, done, truncated, info = env.step(action)
    return obs, rewards, done, truncated, info

def plot_environment(environment_data):
    active_voxels = environment_data > 0.1
    z, y, x = np.where(active_voxels)
    values = environment_data[active_voxels]
    fig = go.Figure(data=go.Volume(
        x=x, y=y, z=z,
        value=values,
        isomin=0.1,
        isomax=values.max(),
        opacity=0.1,
        surface_count=20,
    ))
    fig.update_layout(title="3D View of the Bioprinted Heart")
    return fig

st.title('3D Heart Bioprinting Simulation')
action = st.sidebar.slider("Adjust Action Strength", 0.0, 1.0, 0.5)
if st.sidebar.button('Simulate Step'):
    action_array = np.array([0.5, 0.5, 0.5, 0.5, 1, action], dtype=np.float32)  # Example action array
    obs, rewards, done, truncated, info = simulate_step(action_array)
    st.write("Observation shape:", obs.shape)
    st.write("Reward:", rewards)
    st.write("Done:", done)
    st.write("Info:", info)
    fig = plot_environment(obs[:, :, :, 0])
    st.plotly_chart(fig, use_container_width=True)

st.markdown("""
#### Educational Content
- **Phase 1:** Introduction to 3D bioprinting.
- **Phase 2:** Materials such as myocardium, endocardium, and epicardium.
- **Phase 3:** Bioprinting process and technological challenges.
- **Phase 4:** Future prospects and implications of bioprinting in medicine.
""")

Writing bioprinting_app.py


In [ ]:
# Cell 4: Train the model
!python train_model.py

2024-04-20 22:19:31.897657: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 22:19:31.897726: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 22:19:31.913241: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-20 22:19:36.630649: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using cpu device
-----------------------------
| time/              |      |
|    fps             | 19   |
|    iterations      | 1    |
|    time_elapsed    | 105  |
|    total_timesteps | 2048 |
-----------------------------
Traceback (mos

In [ ]:
# Cell 5: Download the trained model
from google.colab import files
files.download('bioprint_heart_model.zip')

FileNotFoundError: Cannot find file: bioprint_heart_model.zip

In [ ]:
# Cell 6: Save the environment and app files
!mv organ_bioprint_env.py bioprinting_app.py /content/

In [ ]:
# Cell 7: Download the environment and app files
from google.colab import files
files.download('/content/organ_bioprint_env.py')
files.download('/content/bioprinting_app.py')

In [ ]:
# Cell 1: Create organ_bioprint_env.py
%%writefile organ_bioprint_env.py
import numpy as np
import gymnasium as gym
from gymnasium import spaces

class OrganBioprintEnv(gym.Env):
    def __init__(self, resolution=32):
        super().__init__()
        self.resolution = resolution
        self.bioprint_env = np.zeros((resolution, resolution, resolution), dtype=np.float32)
        self.observation_space = spaces.Box(low=0, high=1, shape=(resolution, resolution, resolution), dtype=np.float32)
        self.action_space = spaces.Box(low=-1, high=1, shape=(4,), dtype=np.float32)
        self.target_structure = self.generate_target_structure()
        self.print_head_position = np.array([resolution // 2, resolution // 2, resolution // 2], dtype=np.float32)

    def generate_target_structure(self):
        target = np.zeros((self.resolution, self.resolution, self.resolution), dtype=np.float32)
        r = self.resolution // 2
        grid = np.indices((self.resolution, self.resolution, self.resolution))
        distance = np.sqrt((grid[0] - r)**2 + (grid[1] - r)**2 + (grid[2] - r)**2)
        target[distance < r] = 1
        return target

    def step(self, action):
        dx, dy, dz, extrude = action
        self.print_head_position += np.array([dx, dy, dz], dtype=np.float32) * 2
        self.print_head_position = np.clip(self.print_head_position, 0, self.resolution - 1)
        x, y, z = self.print_head_position.astype(int)
        if extrude > 0:
            self.bioprint_env[x, y, z] = 1

        reward = -np.sum(np.abs(self.bioprint_env - self.target_structure))
        done = np.array_equal(self.bioprint_env, self.target_structure)
        info = {"position": self.print_head_position.tolist()}
        return self.bioprint_env.copy(), reward, done, False, info

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.bioprint_env = np.zeros((self.resolution, self.resolution, self.resolution), dtype=np.float32)
        self.print_head_position = np.array([self.resolution // 2, self.resolution // 2, self.resolution // 2], dtype=np.float32)
        return self.bioprint_env.copy(), {}

# Cell 2: Create bioprinting_app.py
%%writefile bioprinting_app.py
import streamlit as st
import numpy as np
import plotly.graph_objects as go
from organ_bioprint_env import OrganBioprintEnv

env = OrganBioprintEnv()

def simulate_step(action):
    obs, rewards, done, truncated, info = env.step(action)
    return obs, rewards, done, truncated, info

def plot_environment(environment_data):
    active_voxels = environment_data > 0.5
    z, y, x = np.where(active_voxels)
    fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=3, color=z, colorscale='Viridis', opacity=0.8))])
    fig.update_layout(title="3D View of the Bioprinted Structure", scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))
    return fig

st.title('3D Bioprinting Simulation')
dx = st.sidebar.slider("X-axis Movement", -1.0, 1.0, 0.0)
dy = st.sidebar.slider("Y-axis Movement", -1.0, 1.0, 0.0)
dz = st.sidebar.slider("Z-axis Movement", -1.0, 1.0, 0.0)
extrude = st.sidebar.slider("Extrusion", 0.0, 1.0, 0.0)

if st.sidebar.button('Simulate Step'):
    action_array = np.array([dx, dy, dz, extrude], dtype=np.float32)
    obs, rewards, done, truncated, info = simulate_step(action_array)
    st.write("Reward:", rewards)
    st.write("Done:", done)
    fig = plot_environment(obs)
    st.plotly_chart(fig, use_container_width=True)

st.markdown("""
#### Educational Content
- **Phase 1:** Introduction to 3D bioprinting.
- **Phase 2:** Bioprinting process and technological challenges.
- **Phase 3:** Future prospects and implications of bioprinting in medicine.
""")

# Cell 3: Run the Streamlit app
!streamlit run bioprinting_app.py

In [ ]:
# Cell 1: Create organ_bioprint_env.py
%%writefile organ_bioprint_env.py
import numpy as np
import gymnasium as gym
from gymnasium import spaces

class OrganBioprintEnv(gym.Env):
    def __init__(self, resolution=32):
        super().__init__()
        self.resolution = resolution
        self.bioprint_env = np.zeros((resolution, resolution, resolution), dtype=np.float32)
        self.observation_space = spaces.Box(low=0, high=1, shape=(resolution, resolution, resolution), dtype=np.float32)
        self.action_space = spaces.Box(low=-1, high=1, shape=(4,), dtype=np.float32)
        self.target_structure = self.generate_target_structure()
        self.print_head_position = np.array([resolution // 2, resolution // 2, resolution // 2], dtype=np.float32)

    def generate_target_structure(self):
        target = np.zeros((self.resolution, self.resolution, self.resolution), dtype=np.float32)
        r = self.resolution // 4
        cx, cy, cz = self.resolution // 2, self.resolution // 2, self.resolution // 2
        for x in range(self.resolution):
            for y in range(self.resolution):
                for z in range(self.resolution):
                    if (x - cx)**2 + (y - cy)**2 + (z - cz)**2 <= r**2:
                        target[x, y, z] = 1
        return target

    def step(self, action):
        dx, dy, dz, extrude = action
        self.print_head_position += np.array([dx, dy, dz], dtype=np.float32) * 2
        self.print_head_position = np.clip(self.print_head_position, 0, self.resolution - 1)
        x, y, z = self.print_head_position.astype(int)
        if extrude > 0:
            self.bioprint_env[x, y, z] = 1

        reward = -np.sum(np.abs(self.bioprint_env - self.target_structure))
        done = np.array_equal(self.bioprint_env, self.target_structure)
        info = {"position": self.print_head_position.tolist()}
        return self.bioprint_env.copy(), reward, done, False, info

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.bioprint_env = np.zeros((self.resolution, self.resolution, self.resolution), dtype=np.float32)
        self.print_head_position = np.array([self.resolution // 2, self.resolution // 2, self.resolution // 2], dtype=np.float32)
        return self.bioprint_env.copy(), {}

# Cell 2: Create bioprinting_app.py
%%writefile bioprinting_app.py
import streamlit as st
import numpy as np
import plotly.graph_objects as go
from organ_bioprint_env import OrganBioprintEnv

env = OrganBioprintEnv()

def simulate_step(action):
    obs, rewards, done, truncated, info = env.step(action)
    return obs, rewards, done, truncated, info

def plot_environment(environment_data):
    active_voxels = environment_data > 0.5
    z, y, x = np.where(active_voxels)
    fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=3, color=z, colorscale='Viridis', opacity=0.8))])
    fig.update_layout(title="3D View of the Bioprinted Structure", scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))
    return fig

st.title('3D Bioprinting Simulation')
dx = st.sidebar.slider("X-axis Movement", -1.0, 1.0, 0.0)
dy = st.sidebar.slider("Y-axis Movement", -1.0, 1.0, 0.0)
dz = st.sidebar.slider("Z-axis Movement", -1.0, 1.0, 0.0)
extrude = st.sidebar.slider("Extrusion", 0.0, 1.0, 0.0)

if st.sidebar.button('Simulate Step'):
    action_array = np.array([dx, dy, dz, extrude], dtype=np.float32)
    obs, rewards, done, truncated, info = simulate_step(action_array)
    st.write("Reward:", rewards)
    st.write("Done:", done)
    fig = plot_environment(obs)
    st.plotly_chart(fig, use_container_width=True)

st.markdown("""
#### Educational Content
- **Phase 1:** Introduction to 3D bioprinting.
- **Phase 2:** Bioprinting process and technological challenges.
- **Phase 3:** Future prospects and implications of bioprinting in medicine.
""")

# Cell 3: Install dependencies
!pip install streamlit plotly numpy gymnasium

# Cell 4: Run the Streamlit app
!streamlit run bioprinting_app.py

Writing organ_bioprint_env.py
